In [ ]:
import pandas as pd
#mettere .xlsx dopo il nome del documento excel
df = pd.read_excel (r'C:\Users\thomas\Documents\università\valetti\2E.xlsx') 
df.columns = ['s', 'v_norm', 'v_inib', 'conc_inib'] 

msus= input("Enter the measure unit of substrate:")
msuv= input("Enter the measure unit of velocity :")
msui=input("Enter the measure unit of inhibent if present:")
print (df)
print('substrate:'+msus)
print('velocity:'+msuv)
print('inhibent concentration'+msui)
  #bisogna fare in modo che le colonne si rinomino da sole in s, v_norm, v_inib, in modo che si adatti allo script
    # la griglia deve essere in alto a sinistra nel foglio excel, e avere una sola riga di nomi, e in ordine di colonna: substr, v normale, v inibita

In [ ]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 250,
             tools = 'wheel_zoom, reset,pan, box_zoom',
             tooltips = [( 'substrate',   '@s'), ( 'velocity',   '@v')],
            )
output_notebook()

In [ ]:
p = figure(x_axis_label='Concentrazione substrato '+msus,
           y_axis_label='Velocità di reazione '+msuv,
           **param,
           )
p.circle( 's', 'v_norm', source=df )
p.square('s', 'v_inib', source=df) 
show( p )
print('the square values are the inhibited ones')

In [ ]:
df['1/s'] = 1/df['s']
df['1/v_norm'] = 1/df['v_norm']
df['1/v_inib']= 1/df['v_inib']
df


In [ ]:
from scipy.stats import linregress
slope_norm, intercept_norm, r_value, p_value, std_err = linregress( df['1/s'], df['1/v_norm'] )
slope_norm, intercept_norm   
'The slope of the non-inhibited enzyme is {}, and the intercept of the non-inibited enzyme is {}'.format(slope_norm, intercept_norm)

In [ ]:
from scipy.stats import linregress
slope_inib, intercept_inib, r_value, p_value, std_err = linregress( df['1/s'], df['1/v_inib'] )
'The slope of the inhibited enzyme is {}, and the intercept of the inibited enzyme is {}'.format(slope_inib, intercept_inib)  

In [ ]:
from numpy import array 

In [ ]:
#faccio un grafico con le 2 rette per capire il tipo di inibizione
p1 = figure(title = 'Linearizzazione di Lineweaver-Burk',
            x_axis_label= '1/s  ' + msus, 
            y_axis_label= '1/v  '+ msuv,
            **param,
            )
p1.circle( '1/s', '1/v_norm', source=df )
xmax = df['1/s'].max() 
x = array( [-3, xmax] )       
y = x * slope_norm + intercept_norm 
p1.line( x, y, color='green')

p1.square( '1/s', '1/v_inib', source=df)
     
y = x * slope_inib + intercept_inib 
p1.line( x, y, color='red')
show( p1 )

In [ ]:
#calcolo il punto di incontro delle rette, fondamentale per definire il tipo di inibizione
slope_x= slope_norm-slope_inib

intercept_x= -intercept_norm+intercept_inib
intersec_x=intercept_x/slope_x
intersec_x
intersec_y= slope_norm*intersec_x+intercept_norm
intersec_y

p1.triangle( intersec_x, intersec_y, color='black')

show( p1 )
print("The intersection of the 2 lines is at:(" +str(intersec_x)+ ','+str(intersec_y)+'), the black dot stands for the interception point of the 2 lines.')


In [ ]:
#si studia ora l'inibizione con la prima linearizzazione 
#1/v = 1/s * B *(Km/Vmax) + A/Vmax
#equazione di una retta y=mx+n

 #Dove 	1/v = y
#	1/s = x
#	B *(Km/Vmax) = m   
#	A/Vmax = n

#Consideriamo
#m1: m per la reazione senza inibitore
#n1: n per la reazione senza inibitore
#m2: m per la reazione con inibitore
#n2: n per la reazione con inibitore

#inibitore competitivo: n1=n2 perchè A=1
#B = 1 + i/Kei = m2/m1

#inibitore acompetitivo: m1=m2 perchè B=1
#A= 1+ i/Keis = n2/n1

#inibitore non competitivo: m1/n1 =m2/n2 perchè A=B
#A=B=1+1/Ki=n2/n1=m2/m1
#	



In [ ]:
A=intercept_inib/intercept_norm
B=slope_inib/slope_norm
Kei= df['conc_inib']/(B-1)
Keis=df['conc_inib']/(A-1)
Ki=1/(B-1)
compet=print("The facteor A is: "+str( A)+', the factor B is: '+str( B)+'.')

a_compet=print('The Kei is: '+str(Kei)+' '+msus+' the Keis is: ' +str(Keis)+' '+msus+ 'the Ki is: '+str(Ki)+' '+msus)
# tutti i NaN sono dovuti alla mancanza del valore dell inibente, per ora non so come toglierlo

In [ ]:
inib_type = input("According to the graph, which kind inibition do you want to study? competitive, non-competitive, a-competitive:")

if inib_type == 'competitive':
    print(compet)
 


In [ ]:
compet


In [ ]:
#sperimento nuovi grafici

#import matplotlib.pyplot as plt
#import numpy as np
#x = df['1/s']
#y = df['1/v_norm']
#x1= df['1/s']
#y1= df['1/v_inib']
#fig, ax = plt.subplots()
#ax.plot(x, y)
#ax.plot(x1, y1)
#ax.set_ylabel('1/v')
#ax.set_xlabel('1/s')
#ax.set_title('linearisation')
#plt.show()